In [7]:
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import griddata
import netCDF4 as nc
from scipy.interpolate import RegularGridInterpolator
import time
import pickle

import sys
sys.path.append("/home/z5297792/UNSW-MRes/MRes/modules") 
from utils import dopioe, rossby_number, calc_tang_vel, find_directional_radii


In [3]:
def nencioli(u, v, lon, lat, a, b):
    """
    Identify the points in the domain which satisfy the four velocity constraints for eddy detection.

    Parameters:
    - u, v: 2D velocity fields for u and v components
    - lon, lat: Longitude and Latitude matrices
    - mask: Matrix defining sea (1) and land points (0)
    - a, b: Parameters used for constraints

    Returns:
    - eddy_uv: Positions that satisfy the first two constraints (for debugging)
    - eddy_c: Positions satisfying the first three constraints (for debugging)
    - eddy: Positions of the eddy centers with their type (cyclonic=1, anticyclonic=-1) (opposite for me)
    """

    borders = max(a, b) + 1

    # Compute velocity magnitude
    vel = np.sqrt(u**2 + v**2)

    # Initialize arrays for storing eddy centers
    eddy_uv = np.zeros((0, 2))
    eddy_c = np.zeros((0, 2))
    eddy = np.zeros((0, 3))

    # Get domain dimensions
    bound = vel.shape

    # Loop through each latitudinal section
    for i in range(borders, len(v) - borders + 1):
        wrk = v[i, :]  # Latitudinal section of v

        # First constraint: zero crossing in v component
        s = np.sign(wrk)
        indx = np.where(np.diff(s) != 0)[0]
        indx = indx[(indx >= borders) & (indx < len(wrk) - borders)]

        for ii in indx:
            var = 0  # Eddy type (0 = no eddy, 1 = cyclonic, -1 = anticyclonic)
            if wrk[ii] >= 0:  # Anticyclonic
                if wrk[ii - a] > wrk[ii] and wrk[ii + 1 + a] < wrk[ii + 1]:
                    var = -1
            elif wrk[ii] < 0:  # Cyclonic
                if wrk[ii - a] < wrk[ii] and wrk[ii + 1 + a] > wrk[ii + 1]:
                    var = 1

            # Second constraint: u component reversal
            if var != 0:
                if var == -1:
                    if (u[i - a, ii] <= 0 and u[i - a, ii] <= u[i - 1, ii] and
                        u[i + a, ii] >= 0 and u[i + a, ii] >= u[i + 1, ii]):
                        eddy_uv = np.vstack([eddy_uv, [lat[i, ii], lon[i, ii]], [lat[i, ii + 1], lon[i, ii + 1]]])
                    else:
                        var = 0
                elif var == 1:
                    if (u[i - a, ii] >= 0 and u[i - a, ii] >= u[i - 1, ii] and
                        u[i + a, ii] <= 0 and u[i + a, ii] <= u[i + 1, ii]):
                        eddy_uv = np.vstack([eddy_uv, [lat[i, ii], lon[i, ii]], [lat[i, ii + 1], lon[i, ii + 1]]])
                    else:
                        var = 0

                # Third constraint: velocity minimum
                if var != 0:
                    srch = vel[i - b:i + b, ii - b:ii + b + 1]
                    slat = lat[i - b:i + b, ii - b:ii + b + 1]
                    slon = lon[i - b:i + b, ii - b:ii + b + 1]
                    X, Y = np.unravel_index(np.argmin(srch), srch.shape)
                    srch2 = vel[max(i - b + X - 1 - b, 0):min(i - b + X - 1 + b, bound[0]),
                                max(ii - b + Y - 1 - b, 0):min(ii - b + Y - 1 + b, bound[1])]

                    if np.min(srch2) == np.min(srch):
                        eddy_c = np.vstack([eddy_c, [slat[X, Y], slon[X, Y]]])
                    else:
                        var = 0

                # Fourth constraint: vector rotation (simplified version)
                d = a - 1
                if var != 0:
                    # Find indices of the estimated center in the large domain
                    i1, i2 = np.where((lat == slat[X, Y]) & (lon == slon[X, Y]))

                    i1, i2 = int(i1[0]), int(i2[0])
                    
                    # Extract velocities within "a-1" points from the estimated center
                    u_small = u[max(i1 - d, 0):min(i1 + d, bound[0]), max(i2 - d, 0):min(i2 + d, bound[1])]
                    v_small = v[max(i1 - d, 0):min(i1 + d, bound[0]), max(i2 - d, 0):min(i2 + d, bound[1])]
                    
                    # Apply constraint only if there are no NaNs in u_small
                    if not np.isnan(u_small).any():
                        # Boundary velocities
                        u_bound = np.concatenate([u_small[0, :], u_small[1:, -1], u_small[-1, -2::-1], u_small[-2::-1, 0]])
                        v_bound = np.concatenate([v_small[0, :], v_small[1:, -1], v_small[-1, -2::-1], v_small[-2::-1, 0]])

                        # Vector defining which quadrant each boundary vector belongs to
                        quadrants = np.zeros_like(u_bound)
                        quadrants[(u_bound >= 0) & (v_bound >= 0)] = 1
                        quadrants[(u_bound < 0) & (v_bound >= 0)] = 2
                        quadrants[(u_bound < 0) & (v_bound < 0)] = 3
                        quadrants[(u_bound >= 0) & (v_bound < 0)] = 4
                        
                        # Identify the first fourth quadrant vector
                        spin = np.where(quadrants == 4)[0]
                        
                        # Apply the constraint only if the rotation is complete and not all vectors are in the fourth quadrant
                        if spin.size > 0 and spin.size != quadrants.size:
                            # If vectors start in the 4th quadrant, add 4 to all quadrant positions after the first occurrence
                            if spin[0] == 0:
                                spin = np.where(quadrants != 4)[0]
                                spin = spin[0] - 1
                                
                            if not isinstance(spin, np.ndarray):
                                spin = np.array([int(spin)])
                            quadrants[spin[-1] + 1:] += 4
                            
                            # Inspect vector rotation: no consecutive vectors should be more than one quadrant apart
                            # and there should be no backward rotation
                            if not np.any(np.diff(quadrants) > 1) and not np.any(np.diff(quadrants) < 0):
                                eddy = np.vstack([eddy, [slat[X, Y], slon[X, Y], var]])


    # Process eddy results (sorting and removing duplicates)
    eddy = np.unique(eddy, axis=0)
    eddy_uv = np.unique(eddy_uv, axis=0)
    eddy_c = np.unique(eddy_c, axis=0)
    # Adjust for the Southern Hemisphere (flip cyclonic/anticyclonic labels)
    # eddy[eddy[:, 0] < 0, 2] = -eddy[eddy[:, 0] < 0, 2]
    eddy[:, 2] = -eddy[:, 2]
    # Swap for personal preference 
    eddy[:, [0, 1]] = eddy[:, [1, 0]]

    return eddy_uv, eddy_c, eddy


#### Getting the grid

In [4]:
fname = f'/srv/scratch/z3533156/26year_BRAN2020/outer_avg_01461.nc'

dataset = nc.Dataset(fname)

lon_rho = np.transpose(dataset.variables['lon_rho'], axes=(1, 0))
lat_rho = np.transpose(dataset.variables['lat_rho'], axes=(1, 0))
angle = dataset.variables['angle'][0, 0]

def distance(lat1, lon1, lat2, lon2):
    EARTH_RADIUS = 6357
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return EARTH_RADIUS * 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

j_mid = lon_rho.shape[1] // 2
i_mid = lon_rho.shape[0] // 2

dx = distance(lat_rho[:-1, j_mid], lon_rho[:-1, j_mid],
              lat_rho[1:, j_mid], lon_rho[1:, j_mid])
dy = distance(lat_rho[i_mid, :-1], lon_rho[i_mid, :-1],
              lat_rho[i_mid, 1:], lon_rho[i_mid, 1:])

x_grid = np.insert(np.cumsum(dx), 0, 0)
y_grid = np.insert(np.cumsum(dy), 0, 0)
X_grid, Y_grid = np.meshgrid(x_grid, y_grid, indexing='ij')

res = 1  # 1 km resolution
x_new = np.arange(0, x_grid[-1], res)
y_new = np.arange(0, y_grid[-1], res)
X_new, Y_new = np.meshgrid(x_new, y_new, indexing='ij')
new_points = np.column_stack((X_new.ravel(), Y_new.ravel()))

interp_lon = RegularGridInterpolator((x_grid, y_grid), lon_rho,
                                     method='linear', bounds_error=False, fill_value=np.nan)
interp_lat = RegularGridInterpolator((x_grid, y_grid), lat_rho,
                                     method='linear', bounds_error=False, fill_value=np.nan)

lon_new = interp_lon(new_points).reshape(len(x_new), len(y_new))
lat_new = interp_lat(new_points).reshape(len(x_new), len(y_new))



In [5]:
def interpolate_uv(u, v, x_grid, y_grid, X_new, Y_new, angle):
    u_east = np.where(np.abs(u) > 1e30, np.nan, u).astype(float)
    v_north = np.where(np.abs(v) > 1e30, np.nan, v).astype(float)

    u_rot = v_north * np.sin(angle) + u_east * np.cos(angle)
    v_rot = v_north * np.cos(angle) - u_east * np.sin(angle)

    shape_new = X_new.shape
    new_points = np.column_stack((X_new.ravel(), Y_new.ravel()))

    interp_u = RegularGridInterpolator((x_grid, y_grid), u_rot,
                                       method='linear', bounds_error=False, fill_value=np.nan)
    interp_v = RegularGridInterpolator((x_grid, y_grid), v_rot,
                                       method='linear', bounds_error=False, fill_value=np.nan)

    u_new = interp_u(new_points).reshape(shape_new)
    v_new = interp_v(new_points).reshape(shape_new)

    return u_new, v_new

def create_velocity_dic(start_day, num_days, x_grid, y_grid, X_new, Y_new, angle):
    ROMS_velocity_dic = {}
    for day in range(start_day, start_day + num_days + 1):
        fnumber = 1461 + ((day - 1462) // 30)*30
        fname = f'/srv/scratch/z3533156/26year_BRAN2020/outer_avg_{fnumber:05}.nc'
        dataset = nc.Dataset(fname)
        u_east = np.transpose(dataset['u_eastward'][:].data, axes=(3, 2, 1, 0))[:, :, -1, :].squeeze()
        v_north = np.transpose(dataset['v_northward'][:].data, axes=(3, 2, 1, 0))[:, :, -1, :].squeeze()
        ocean_time = dataset.variables['ocean_time'][:].data / 86400
        t = np.where(day==ocean_time)[0][0]
        if t.size != 0:
            u, v = u_east[:, :, t], v_north[:, :, t]
            u_new, v_new = interpolate_uv(u, v, x_grid, y_grid, X_new, Y_new, angle)
            ROMS_velocity_dic[day] = {'u': u_new, 'v': v_new, 'fname': fname, 't': t}
        if day % 20 == 0:
            print(day)
    return ROMS_velocity_dic

tic = time.perf_counter()

start_day, num_days = 5480, 5170     # last valid day 10650 (10641 excluding the small end file)
ROMS_velocity_dic = create_velocity_dic(start_day, num_days, x_grid, y_grid, X_new, Y_new, angle)

with open(f"/srv/scratch/z5297792/Chapter2/Eddy_Detection_Data/ROMS_velocity_dic_{start_day}_{start_day+num_days}.pkl", "wb") as f:
    pickle.dump(ROMS_velocity_dic, f)

toc = time.perf_counter()
print(f"Elapsed time: {toc - tic:.4f} seconds")

# start_day, num_days = 5480, 5170     # last valid day 10650 (10641 excluding the small end file)
# with open(f"/srv/scratch/z5297792/Chapter2/Eddy_Detection_Data/ROMS_velocity_dic_{start_day}_{start_day+num_days}.pkl", "rb") as f:
#     ROMS_velocity_dic = pickle.load(f)


EOFError: Ran out of input

In [6]:
def build_nenc_dataframe(ROMS_velocity_dic, X_new, Y_new, lon_new, lat_new):
    rows = []
    for day, data in ROMS_velocity_dic.items():
        u0, v0 = data['u'], data['v']
        # nencioli returns a tuple; take index 2 for neddy
        neddy = nencioli(u0.T, v0.T, X_new.T, Y_new.T, 4, 3)[2]
        # Sort so that the highest second-column value comes first
        neddy = neddy[neddy[:, 1].argsort()[::-1]]
        
        for idx, (nxc0, nyc0, cyc_indicator) in enumerate(neddy):
            cyc_value = 'CE' if cyc_indicator == 1 else 'AE'
            nic_idx, njc_idx = np.where((X_new == nxc0) & (Y_new == nyc0))
            if nic_idx.size:
                nic0, njc0 = nic_idx[0], njc_idx[0]
            else:
                nic0, njc0 = np.nan, np.nan
            
            rows.append({
                'Eddy': idx,
                'Day': day,
                'Cyc': cyc_value,
                'nLon': lon_new[nic0, njc0],
                'nLat': lat_new[nic0, njc0],
                'nxc': nxc0,
                'nyc': nyc0,
                'nic': nic0,
                'njc': njc0
            })
        if day % 20 == 0:
            print(day)
    
    return pd.DataFrame(rows)

tic = time.perf_counter()

df_nenc = build_nenc_dataframe(ROMS_velocity_dic, X_new, Y_new, lon_new, lat_new) # bit over 4hrs

df_nenc.to_pickle(f"/srv/scratch/z5297792/Chapter2/Eddy_Detection_Data/df_nenc_{start_day}_{start_day+num_days}.pkl")

toc = time.perf_counter()
print(f"Elapsed time: {toc - tic:.4f} seconds")


NameError: name 'ROMS_velocity_dic' is not defined

In [ ]:
def update_nenc_dataframe(df_nenc, ROMS_velocity_dic, X_new, Y_new, r=30):

    # DOPIOE wont work if too close to boundary
    x_new = X_new[:, 0]
    y_new = Y_new[0, :]
    dx = np.max(np.diff(x_new))  # spacing in x-direction
    dy = np.max(np.diff(y_new))  # spacing in y-direction
    cell_size = np.max([dx, dy])        # average cell size in Euclidean units
    margin = int(np.ceil(r / cell_size))    

    df_data = df_nenc.copy()
    for day in df_data['Day'].unique():
        for e in df_data[df_data['Day'] == day]['Eddy'].unique():
            # Get eddy location from the first row for this day/eddy combination.
            row = df_data[(df_data['Day'] == day) & (df_data['Eddy'] == e)].iloc[0]
            nxc, nyc, cyc = row['nxc'], row['nyc'], row['Cyc']

            R_grid = np.hypot(nxc - X_new, nyc - Y_new)
            ic, jc = map(int, np.unravel_index(np.argmin(R_grid), R_grid.shape))

            if (ic < margin or ic >= X_new.shape[0] - margin or
                jc < margin or jc >= X_new.shape[1] - margin):
                xc, yc, w, Q, Rc, psi0, Ro = np.nan, np.nan, np.nan, np.array([[np.nan, np.nan],
                                                           [np.nan, np.nan]]), np.nan, np.nan, np.nan
            else:
                ut, vt = ROMS_velocity_dic[day]['u'], ROMS_velocity_dic[day]['v']
    
                # horizontal transect (constant y = y[jc])
                x_mask = np.abs(x_new - nxc) < r
                x1 = x_new[x_mask]
                y1 = np.full_like(x1, y_new[jc])
                u1 = ut[x_mask, jc]
                v1 = vt[x_mask, jc]
            
                # vertical transect (constant x = x[ic])
                y_mask = np.abs(y_new - nyc) < r
                y2 = y_new[y_mask]
                x2 = np.full_like(y2, x_new[ic])
                u2 = ut[ic, y_mask]
                v2 = vt[ic, y_mask]
    
                xc, yc, w, Q, _, psi0, _ = dopioe(x1, y1, u1, v1, x2, y2, u2, v2)
    
                cyc_DOPIOE = 'CE' if w < 0 else 'AE'
    
                if (cyc_DOPIOE != cyc) or (np.hypot(nxc - xc, nyc - yc) > 50):
                    xc, yc, w, Q, Rc, psi0, Ro = np.nan, np.nan, np.nan, np.array([[np.nan, np.nan],
                                                               [np.nan, np.nan]]), np.nan, np.nan, np.nan
                else:
                    w *= 1e-3 # to s^-1
                    Ro = rossby_number(w, yc)
        
                    radii = find_directional_radii(ut, vt, X_new, Y_new, xc, yc, calc_tang_vel)
                    Rc = np.mean([radii['up'], radii['right'], radii['down'], radii['left']])
                
            # Update DataFrame for this day/eddy.
            update_mask = (df_data['Day'] == day) & (df_data['Eddy'] == e)
            df_data.loc[update_mask, 'xc'] = xc
            df_data.loc[update_mask, 'yc'] = yc
            df_data.loc[update_mask, 'w'] = w
            df_data.loc[update_mask, 'Q11'] = Q[0, 0]
            df_data.loc[update_mask, 'Q12'] = Q[1, 0]
            df_data.loc[update_mask, 'Q22'] = Q[1, 1]
            df_data.loc[update_mask, 'Rc'] = Rc
            df_data.loc[update_mask, 'psi0'] = psi0
            df_data.loc[update_mask, 'Ro'] = Ro

        if day % 20 == 0:
            print(day)

    return df_data

tic = time.perf_counter()

df_data = update_nenc_dataframe(df_nenc, ROMS_velocity_dic, X_new, Y_new)

del df_nenc, ROMS_velocity_dic

df_data.to_pickle(f"/srv/scratch/z5297792/Chapter2/Eddy_Detection_Data/df_data_{start_day}_{start_day+num_days}.pkl") # 2 hrs

toc = time.perf_counter()
print(f"Elapsed time: {toc - tic:.4f} seconds")


